# 最大熵模型 

### 绘图示范

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
x = np.linspace(0,2*np.pi,100)
y = np.sin(x)

In [ ]:
plt.plot(x,y)

In [ ]:
import sympy 
from sympy import * 

x,y,z,a,b,c = symbols('x,y,z,a,b,c')

In [ ]:
sympy.plot(sin(x),(x,0,2*pi))

### 语料分词

In [107]:
import numpy as np 
import pandas as pd 

df = pd.read_csv('weekend_activity',sep = '\t',encoding = 'utf-8')
df

,label,text
0,户外活动,天气晴朗，气温凉爽，心情愉快
1,户外活动,天气晴朗，气温炎热，心情下雨
2,户外活动,天气晴朗，气温寒冷，心情愉快
3,户外活动,天气晴朗，气温凉爽，心情晴朗
4,宅在家里,天气晴朗，气温炎热，心情忧伤
5,宅在家里,天气下雨，气温凉爽，心情多云
6,户外活动,天气多云，气温凉爽，心情忧伤
7,户外活动,天气下雨，气温凉爽，心情愉快
8,户外活动,天气下雨，气温寒冷，心情多云
9,宅在家里,天气下雨，气温寒冷，心情下雨


In [108]:
# 对label转化成 0,1等数字类别
dfdata = df.copy()
dfdata['label'] = pd.get_dummies(dfdata['label'])[u'户外活动']

In [109]:
import jieba 
texts = dfdata['text'].tolist()
seglist = [list(filter(lambda k:len(k)>1, jieba.lcut(x))) for x in texts]
seglist

[['天气晴朗', '气温', '凉爽', '心情愉快'],
 ['天气晴朗', '气温', '炎热', '心情', '下雨'],
 ['天气晴朗', '气温', '寒冷', '心情愉快'],
 ['天气晴朗', '气温', '凉爽', '心情', '晴朗'],
 ['天气晴朗', '气温', '炎热', '心情', '忧伤'],
 ['天气', '下雨', '气温', '凉爽', '心情', '多云'],
 ['天气', '多云', '气温', '凉爽', '心情', '忧伤'],
 ['天气', '下雨', '气温', '凉爽', '心情愉快'],
 ['天气', '下雨', '气温', '寒冷', '心情', '多云'],
 ['天气', '下雨', '气温', '寒冷', '心情', '下雨'],
 ['天气晴朗', '气温', '寒冷', '心情', '忧伤'],
 ['天气', '多云', '气温', '凉爽', '心情愉快'],
 ['天气', '多云', '气温', '凉爽', '心情', '忧伤'],
 ['天气', '下雨', '气温', '凉爽', '心情愉快']]

In [110]:
import pkuseg
seg = pkuseg.pkuseg() 
texts = dfdata['text'].tolist()
seglist = [list(filter(lambda k:len(k)>1, seg.cut(x))) for x in texts]
seglist

loading model
finish


[['天气', '晴朗', '气温', '凉爽', '心情', '愉快'],
 ['天气', '晴朗', '气温', '炎热', '心情', '下雨'],
 ['天气', '晴朗', '气温', '寒冷', '心情', '愉快'],
 ['天气', '晴朗', '气温', '凉爽', '心情', '晴朗'],
 ['天气', '晴朗', '气温', '炎热', '心情', '忧伤'],
 ['天气', '下雨', '气温', '凉爽', '心情', '多云'],
 ['天气', '多云', '气温', '凉爽', '心情', '忧伤'],
 ['天气', '下雨', '气温', '凉爽', '心情', '愉快'],
 ['天气', '下雨', '气温', '寒冷', '心情', '多云'],
 ['天气', '下雨', '气温', '寒冷', '心情', '下雨'],
 ['天气', '晴朗', '气温', '寒冷', '心情', '忧伤'],
 ['天气', '多云', '气温', '凉爽', '心情', '愉快'],
 ['天气', '多云', '气温', '凉爽', '心情', '忧伤'],
 ['天气', '下雨', '气温', '凉爽', '心情', '愉快']]

### 构建词典

In [135]:
import gensim
import logging
from gensim import corpora, models, similarities

#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
dictionary = corpora.Dictionary(seglist)
dictionary.save('activity.dict')  # store the dictionary

# 建立完整字典
dictionary.token2id

{'凉爽': 0,
 '天气': 1,
 '心情': 2,
 '愉快': 3,
 '晴朗': 4,
 '气温': 5,
 '下雨': 6,
 '炎热': 7,
 '寒冷': 8,
 '忧伤': 9,
 '多云': 10}

In [93]:
# 重点词典，去除文档频率2次以下和0.8以上的词
dictionary.filter_extremes(no_below=2,no_above=0.8,keep_n = 100000,keep_tokens = None)
dictionary.token2id

{'凉爽': 0, '愉快': 1, '晴朗': 2, '下雨': 3, '炎热': 4, '寒冷': 5, '忧伤': 6, '多云': 7}

In [83]:
hashdict = HashDictionary(seglist,debug=True,id_range=5)  
print(hashdict.token2id)
hashdict

{'凉爽': 0, '天气': 3, '心情': 0, '愉快': 0, '晴朗': 0, '气温': 3, '下雨': 0, '炎热': 2, '寒冷': 4, '忧伤': 0, '多云': 4}


{0: {'下雨', '凉爽', '心情', '忧伤', '愉快', '晴朗'},
 1: set(),
 2: {'炎热'},
 3: {'天气', '气温'},
 4: {'多云', '寒冷'}}

### 词袋模型

In [128]:
corpus

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)],
 [(1, 1), (2, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (8, 1)],
 [(0, 1), (1, 1), (2, 1), (4, 2), (5, 1)],
 [(1, 1), (2, 1), (4, 1), (5, 1), (7, 1), (9, 1)],
 [(0, 1), (1, 1), (2, 1), (5, 1), (6, 1), (10, 1)],
 [(0, 1), (1, 1), (2, 1), (5, 1), (9, 1), (10, 1)],
 [(0, 1), (1, 1), (2, 1), (3, 1), (5, 1), (6, 1)],
 [(1, 1), (2, 1), (5, 1), (6, 1), (8, 1), (10, 1)],
 [(1, 1), (2, 1), (5, 1), (6, 2), (8, 1)],
 [(1, 1), (2, 1), (4, 1), (5, 1), (8, 1), (9, 1)],
 [(0, 1), (1, 1), (2, 1), (3, 1), (5, 1), (10, 1)],
 [(0, 1), (1, 1), (2, 1), (5, 1), (9, 1), (10, 1)],
 [(0, 1), (1, 1), (2, 1), (3, 1), (5, 1), (6, 1)]]

In [124]:
len(dictionary)

11

In [130]:
# 建立bag of word稀疏矩阵
corpus = [dictionary.doc2bow(text) for text in seglist]

# 词袋模型
bow_array = gensim.matutils.corpus2dense(corpus, num_terms=len(dictionary)).T
bow_array

array([[1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
       [0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0.],
       [1., 1., 1., 0., 2., 1., 0., 0., 0., 0., 0.],
       [0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0.],
       [1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1.],
       [1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1.],
       [1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1.],
       [0., 1., 1., 0., 0., 1., 2., 0., 1., 0., 0.],
       [0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0.],
       [1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1.],
       [1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1.],
       [1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0.]], dtype=float32)

### TF-IDF权重向量

In [136]:
# TF_IDF权重向量
tfidf = models.TfidfModel(corpus)
tfidf_array = gensim.matutils.corpus2dense(list(tfidf[corpus]),num_terms=len(dictionary)).T
tfidf_array[0:5]

array([[0.38698328, 0.        , 0.        , 0.71199834, 0.58592004,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.3707672 ,
        0.        , 0.3707672 , 0.85150653, 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.5627558 , 0.4631049 ,
        0.        , 0.        , 0.        , 0.6847187 , 0.        ,
        0.        ],
       [0.31357917, 0.        , 0.        , 0.        , 0.9495621 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.3437973 ,
        0.        , 0.        , 0.7895673 , 0.        , 0.50831777,
        0.        ]], dtype=float32)

### LSI主题模型

In [143]:
# LSI主题模型 3个主题
lsi = models.lsimodel.LsiModel(corpus=corpus, id2word=dictionary, num_topics=3)
lsi.print_topics()

[(0,
  '-0.483*"天气" + -0.483*"心情" + -0.483*"气温" + -0.284*"凉爽" + -0.244*"下雨" + -0.241*"晴朗" + -0.176*"愉快" + -0.172*"多云" + -0.134*"寒冷" + -0.132*"忧伤"'),
 (1,
  '-0.725*"晴朗" + 0.558*"下雨" + 0.275*"多云" + -0.197*"忧伤" + -0.170*"炎热" + 0.117*"凉爽" + 0.060*"愉快" + 0.039*"寒冷" + -0.014*"天气" + -0.014*"心情"'),
 (2,
  '-0.548*"凉爽" + 0.528*"下雨" + 0.413*"寒冷" + -0.362*"多云" + 0.201*"晴朗" + -0.175*"忧伤" + -0.156*"愉快" + 0.153*"炎热" + 0.018*"心情" + 0.018*"气温"')]

In [144]:
# LSI主题模型
lsi_array = gensim.matutils.corpus2dense(list(lsi[corpus]),num_terms=3).T
lsi_array

array([[-2.1516454 , -0.5906748 , -0.44940165],
       [-2.00055   , -0.37951356,  0.9351639 ],
       [-2.0019393 , -0.6682296 ,  0.51109177],
       [-2.2166624 , -1.3755469 , -0.09214475],
       [-1.8888453 , -1.1351495 ,  0.2322983 ],
       [-2.1506233 ,  0.90750366, -0.32825732],
       [-2.0389187 ,  0.15186773, -1.0311229 ],
       [-2.154523  ,  0.6922803 , -0.122334  ],
       [-2.0009174 ,  0.8299489 ,  0.6322361 ],
       [-2.0727115 ,  1.1128085 ,  1.522484  ],
       [-1.9581293 , -0.92578256,  0.4925507 ],
       [-2.0827289 ,  0.4094207 , -1.0125818 ],
       [-2.0389187 ,  0.15186773, -1.0311229 ],
       [-2.154523  ,  0.6922803 , -0.122334  ]], dtype=float32)

### LDA主题模型

In [157]:
# lda主题模型
lda = models.ldamodel.LdaModel(corpus=corpus,id2word=dictionary, num_topics=5,
                              update_every=100, chunksize=1000, passes=50)
lda.print_topics()

[(0,
  '0.166*"天气" + 0.166*"心情" + 0.166*"气温" + 0.120*"凉爽" + 0.096*"晴朗" + 0.087*"愉快" + 0.070*"忧伤" + 0.053*"多云" + 0.037*"寒冷" + 0.036*"下雨"'),
 (1,
  '0.091*"下雨" + 0.091*"天气" + 0.091*"心情" + 0.091*"气温" + 0.091*"寒冷" + 0.091*"多云" + 0.091*"凉爽" + 0.091*"晴朗" + 0.091*"忧伤" + 0.091*"愉快"'),
 (2,
  '0.211*"炎热" + 0.156*"晴朗" + 0.140*"天气" + 0.140*"心情" + 0.140*"气温" + 0.116*"下雨" + 0.020*"忧伤" + 0.019*"寒冷" + 0.019*"凉爽" + 0.019*"多云"'),
 (3,
  '0.091*"下雨" + 0.091*"天气" + 0.091*"气温" + 0.091*"心情" + 0.091*"寒冷" + 0.091*"多云" + 0.091*"凉爽" + 0.091*"晴朗" + 0.091*"忧伤" + 0.091*"愉快"'),
 (4,
  '0.208*"下雨" + 0.158*"天气" + 0.158*"心情" + 0.158*"气温" + 0.109*"寒冷" + 0.109*"多云" + 0.059*"凉爽" + 0.010*"晴朗" + 0.010*"忧伤" + 0.010*"愉快"')]

In [158]:
# lda主题模型
lda_array  = gensim.matutils.corpus2dense(list(lda[corpus]),num_terms=5).T
lda_array

array([[0.8848183 , 0.02859069, 0.02902525, 0.02859069, 0.02897502],
       [0.02944331, 0.02859281, 0.8837875 , 0.02859281, 0.02958359],
       [0.8843619 , 0.02860033, 0.02901451, 0.02860033, 0.02942291],
       [0.88460606, 0.02859017, 0.02924196, 0.02859017, 0.02897162],
       [0.57300234, 0.02859997, 0.34079605, 0.02859997, 0.02900169],
       [0.02968639, 0.02859565, 0.02892356, 0.02859566, 0.8841987 ],
       [0.8846884 , 0.02859597, 0.02882653, 0.02859598, 0.02929314],
       [0.8833896 , 0.02859949, 0.02921629, 0.0285995 , 0.0301951 ],
       [0.02922238, 0.02859006, 0.02892232, 0.02859006, 0.88467515],
       [0.02918382, 0.02858777, 0.02898027, 0.02858777, 0.88466036],
       [0.8843264 , 0.02860162, 0.02901511, 0.02860162, 0.02945526],
       [0.8846726 , 0.02859469, 0.02882619, 0.0285947 , 0.02931182],
       [0.8847076 , 0.02859597, 0.02882647, 0.02859597, 0.02927396],
       [0.8833676 , 0.02859949, 0.02921631, 0.0285995 , 0.03021707]],
      dtype=float32)

### Word2Vec

In [219]:
from gensim.models import word2vec

# 训练词向量
model = word2vec.Word2Vec(seglist, hs=1,min_count=1,window=2,size=5,iter = 10) 
# 保存词向量
model.wv.save_word2vec_format('vec.txt',binary = False)
# 查看词向量
model.wv['心情']

array([ 0.0078106 , -0.05728721,  0.08298404,  0.03091619,  0.09410242],
      dtype=float32)

In [220]:
# 寻找最相似的词
model.wv.most_similar(['气温'],topn =3)

[('晴朗', 0.5466448068618774),
 ('凉爽', 0.507328987121582),
 ('炎热', 0.42405781149864197)]

In [221]:
# 计算余弦相似度
model.wv.similarity('多云', '下雨')

0.7826325

In [251]:
# 计算余弦相似度
model.wv.similarity('多云', '寒冷')

-0.33808666

### FastText

In [277]:
from gensim.models import FastText

model = FastText(seglist, size=5, window=2, min_count=1,iter=250)
# 保存词向量
model.wv.save_word2vec_format('vec.txt',binary = False)
# 查看词向量
model.wv['心情']

array([-0.17457816,  0.04080092,  0.03789375, -0.01083995, -0.14343518],
      dtype=float32)

In [278]:
# 寻找最相似的词
model.wv.most_similar(['气温'],topn =3)

[('多云', 0.9617383480072021),
 ('愉快', 0.9481709599494934),
 ('炎热', 0.9225022196769714)]

In [279]:
# 计算余弦相似度
model.wv.similarity('多云', '下雨')

0.75107634

In [280]:
# 计算余弦相似度
model.wv.similarity('多云', '寒冷')

-0.07756445

In [282]:
# 计算短语的词向量
model.wv['天气多云转晴']

array([-0.34318325,  0.07374957, -0.10595525,  0.02992013, -0.00185205],
      dtype=float32)

In [284]:
# 计算短语的词向量
model.wv['天气/多云/转晴']

array([-0.34318325,  0.07374957, -0.10595525,  0.02992013, -0.00185205],
      dtype=float32)

In [286]:
# 计算短语的词向量
model.wv['天气/多云']

array([-0.23769134,  0.14257799, -0.10763248, -0.06165032, -0.09630556],
      dtype=float32)

In [302]:
# 计算句子的词向量
model.wv['天气多云转晴，心情晴转多云。']

array([-0.34318325,  0.07374957, -0.10595525,  0.02992013, -0.00185205],
      dtype=float32)

In [329]:
# 计算句子的词向量
model.wv['心情晴转下雨。']

array([-0.0457298 ,  0.14053859,  0.20732215,  0.07776733, -0.22281381],
      dtype=float32)

In [305]:
# 计算句子的相似度
model.wv.similarity('天气多云转晴，心情晴转多云。','天气晴转多云，心情多云转晴。')

1.0

In [332]:
# 计算句子的相似度
model.wv.similarity('天气多云转晴，心情晴转多云.','心情晴转下雨。')

0.05348643

### 最大熵模型

In [ ]:
# -*- coding:utf-8 -*-
# Filename: maxent.py
# Author：hankcs
# Date: 2015/10/2 23:23
import sys
import math
from collections import defaultdict

class MaxEnt:
    def __init__(self):
        self._samples = [] 	    #样本集, 元素是[y,x1,x2,...,xn]的元组
        self._Y = set([])       #标签集合,相当于去重之后的y
        self._numXY = defaultdict(int)  #Key是(xi,yi)对，Value是count(xi,yi)
        self._N = 0		    #样本数量
        self._n = 0		    #特征对(xi,yi)总数量
        self._xyID = {}	    #对(x,y)对做的顺序编号(ID), Key是(xi,yi)对,Value是ID
        self._C = 0		    #样本最大的特征数量,用于求参数时的迭代，见IIS原理说明
        self._ep_ = []		#样本分布的特征期望值
        self._ep = []		#模型分布的特征期望值
        self._w = []		#对应n个特征的权值
        self._lastw = []	#上一轮迭代的权值
        self._EPS = 0.01	#判断是否收敛的阈值
        
    def load_data(self, filename):
        for line in open(filename, "r"):
            sample = line.strip().split("\t")
            if len(sample) < 2: #至少：标签+一个特征
                continue
            y = sample[0]
            X = sample[1:]
            self._samples.append(sample) #label + features
            self._Y.add(y)      #label
            for x in set(X):    #set给X去重
                self._numXY[(x, y)] += 1
                
    def _initparams(self):
        self._N = len(self._samples)
        self._n = len(self._numXY)
        self._C = max([len(sample) - 1 for sample in self._samples])
        self._w = [0.0] * self._n
        self._lastw = self._w[:]
        self._sample_ep()
        
    def _convergence(self):
        for w, lw in zip(self._w, self._lastw):
            if math.fabs(w - lw) >= self._EPS:
                return False
        return True
    
    def _sample_ep(self):
        self._ep_ = [0.0] * self._n
        #计算方法参见公式(20)
        for i, xy in enumerate(self._numXY):
            self._ep_[i] = self._numXY[xy] * 1.0 / self._N
            self._xyID[xy] = i
            
    def _zx(self, X):
        #calculate Z(X), 计算方法参见公式(15)
        ZX = 0.0
        for y in self._Y:
            sum = 0.0
            for x in X:
                if (x, y) in self._numXY:
                    sum += self._w[self._xyID[(x, y)]]
            ZX += math.exp(sum)
        return ZX
    
    def _pyx(self, X):
        #calculate p(y|x), 计算方法参见公式(22)
        ZX = self._zx(X)
        results = []
        for y in self._Y:
            sum = 0.0
            for x in X:
                if (x, y) in self._numXY: #这个判断相当于指示函数的作用
                    sum += 	self._w[self._xyID[(x, y)]]
            pyx = 1.0 / ZX * math.exp(sum)
            results.append((y, pyx))
        return results
    
    def _model_ep(self):
        self._ep = [0.0] * self._n
        #参见公式(21)
        for sample in self._samples:
            X = sample[1:]
            pyx = self._pyx(X)
            for y, p in pyx:
                for x in X:
                    if (x, y) in self._numXY:
                        self._ep[self._xyID[(x, y)]] += p * 1.0 / self._N
                        
    def train(self, maxiter = 1000):
        self._initparams()
        for i in range(0, maxiter):
            print("Iter:%d..."%i)
            self._lastw = self._w[:] #保存上一轮权值
            self._model_ep()
            #更新每个特征的权值
            for i, w in enumerate(self._w):
                #参考公式(19)
                self._w[i] += 1.0 / self._C * math.log(self._ep_[i] / self._ep[i])
            print(self._w)
            #检查是否收敛	
            if self._convergence():
                break
                
    def predict(self, input):
        X = input.strip().split("\t")
        prob = self._pyx(X)
        return prob

In [ ]:
math.fabs(0.001)

In [ ]:
abs(0.001)

In [ ]:
max([len(sample) - 1 for sample in maxent._samples])

In [ ]:
maxent = MaxEnt()
maxent.load_data('train_data')
maxent.train()
print(maxent.predict("Rainy\tHappy\tDry"))
#sys.exit(0)

In [ ]:
maxent = MaxEnt()
maxent.load_data('data.txt')
maxent.train()
print(maxent.predict("sunny\thot\thigh\tFALSE"))
print(maxent.predict("overcast\thot\thigh\tFALSE"))
print(maxent.predict("sunny\tcool\thigh\tTRUE"))
#sys.exit(0)

In [ ]:
# -*- coding:utf-8 -*-
# Filename: maxent.py
# Author：hankcs
# Date: 2015/10/2 23:23
import sys
import math
from collections import defaultdict

class MaxEnt:
    def __init__(self):
        self._samples = [] 	    #样本集, 元素是[y,x1,x2,...,xn]的元组
        self._Y = set([])       #标签集合,相当于去重之后的y
        self._numXY = defaultdict(int)  #Key是(xi,yi)对，Value是count(xi,yi)
        self._N = 0		    #样本数量
        self._n = 0		    #特征对(xi,yi)总数量
        self._xyID = {}	    #对(x,y)对做的顺序编号(ID), Key是(xi,yi)对,Value是ID
        self._C = 0		    #样本最大的特征数量,用于求参数时的迭代，见IIS原理说明
        self._ep_ = []		#样本分布的特征期望值
        self._ep = []		#模型分布的特征期望值
        self._w = []		#对应n个特征的权值
        self._lastw = []	#上一轮迭代的权值
        self._EPS = 0.01	#判断是否收敛的阈值
        
    def load_data(self, filename):
        for line in open(filename, "r"):
            sample = line.strip().split("\t")
            if len(sample) < 2: #至少：标签+一个特征
                continue
            y = sample[0]
            X = sample[1:]
            self._samples.append(sample) #label + features
            self._Y.add(y)      #label
            for x in set(X):    #set给X去重
                self._numXY[(x, y)] += 1
                
    def _initparams(self):
        self._N = len(self._samples)
        self._n = len(self._numXY)
        self._C = max([len(sample) - 1 for sample in self._samples])
        self._w = [0.0] * self._n
        self._lastw = self._w[:]
        self._sample_ep()
        
    def _convergence(self):
        for w, lw in zip(self._w, self._lastw):
            if abs(w - lw) >= self._EPS:
                return False
        return True
    
    def _sample_ep(self):
        self._ep_ = [0.0] * self._n
        #计算方法参见公式(20)
        for i, xy in enumerate(self._numXY):
            self._ep_[i] = self._numXY[xy] * 1.0 / self._N
            self._xyID[xy] = i
            
    def _zx(self, X):
        #calculate Z(X), 计算方法参见公式(15)
        ZX = 0.0
        for y in self._Y:
            sum = 0.0
            for x in X:
                if (x, y) in self._numXY:
                    sum += self._w[self._xyID[(x, y)]]
            ZX += math.exp(sum)
        return ZX
    
    def _pyx(self, X):
        #calculate p(y|x), 计算方法参见公式(22)
        ZX = self._zx(X)
        results = []
        for y in self._Y:
            sum = 0.0
            for x in X:
                if (x, y) in self._numXY: #这个判断相当于指示函数的作用
                    sum += 	self._w[self._xyID[(x, y)]]
            pyx = 1.0 / ZX * math.exp(sum)
            results.append((y, pyx))
        return results
    
    def _model_ep(self):
        self._ep = [0.0] * self._n
        #参见公式(21)
        for sample in self._samples:
            X = sample[1:]
            pyx = self._pyx(X)
            for y, p in pyx:
                for x in X:
                    if (x, y) in self._numXY:
                        self._ep[self._xyID[(x, y)]] += p * 1.0 / self._N
                        
    def train(self, maxiter = 1000):
        self._initparams()
        for i in range(0, maxiter):
            print("Iter:%d..."%i)
            self._lastw = self._w[:] #保存上一轮权值
            self._model_ep()
            #更新每个特征的权值
            for i, w in enumerate(self._w):
                #参考公式(19)
                self._w[i] += 1.0 / self._C * math.log(self._ep_[i] / self._ep[i])
            print(self._w)
            #检查是否收敛	
            if self._convergence():
                break
                
    def predict(self, input):
        X = input.strip().split("\t")
        prob = self._pyx(X)
        return prob

In [ ]:
df.to_csv('train_data',sep = '\t',index = None, header = None )